# Markdown

## Part 1.1 (17 points): 
Use [markdown](https://www.markdownguide.org/cheat-sheet/) to create your own brief wikipedia-esque description of any subject of interest. 

Your mini-wiki page must include:
- three headers: a title, subtitle and subsubtitle (the #, ##, ### syntax)
- an embedded image from a web address (use an [image hosting site](https://makeawebsitehub.com/free-photo-hosting/) if you'd like to upload your own)
    - note gradescope may not render the image, if it shows on jupyter locally we'll award credit
- a table of size at least 3 rows x 3 columns (needn't be correct, but must make sense)
- a list
- a link to another website

Please be brief in your text.  Aim for roughly 3 sentences total of text.  We won't grade based on content, but keep it appropriate for class.  If you make the grader smile, no extra credit will be awarded beyond the satisfaction of having made somebody's day better :)


# Here's a Title
## And Here's a Subtitle

Here's a description of something interesting that I have to say. Something **important**, which is why I'm expanding on that thought.
- This is a list with *several* items
- Below is a picture I found funny
- And here's a link to [some website](https://www.website.com/)
<img src = https://www.ramsoft.com/wp-content/uploads/2021/12/found-this-humeurs.jpg, width = 400>

### Here's a Sub-subtitle and a Table

| A   	| B  	| C     	| D      	|
|-----	|----	|-------	|--------	|
| I   	| 1  	| Ace   	| Red    	|
| II  	| 4  	| Jack  	| Green  	|
| III 	| 9  	| Queen 	| Blue   	|
| VI  	| 16 	| King  	| Yellow 	|

# Part 1.2

In [blackjack](https://en.wikipedia.org/wiki/Blackjack), players are given two cards from a [standard deck](https://en.wikipedia.org/wiki/Standard_52-card_deck#Composition) of cards and add their values together.  Values are assigned to cards as:
- number cards use their own value
- Jack, Queen, King cards all have a value of 10
- **aces, in this problem will always have a value of 1**

Players may then choose to take as many cards as they'd like, one at a time, to increase their hand's sum.  The goal is to produce a value as high as possible without exceeding 21.  When a player's hand sum increases beyond 21 they are said to 'bust' and they lose the game.

The key to playing well is knowing when to stop taking cards.  We'll explore this issue by writing a program to have the computer simulate many hands of blackjack and count how often they bust.


## Part 1.2.1 (10 points)
Build a `draw_value()` function which accepts no inputs and returns the value (an integer from 1 to 10) of some card drawn from a standard deck of cards.  `random.choices()`, and its `weights` parameter may be helpful here.  Assume that you draw cards with replacement, so that its possible to draw the same card many times.  

(In practice, this isn't far from the truth as real casinos often draw from a pile of cards containing multiple, shuffled decks to mitigate the effectiveness of [card counting](https://en.wikipedia.org/wiki/Card_counting)).


In [1]:
import random as rnd

def draw_value():
    """ draws a random card from a pile, only numbered cards 1 - 10.

    returns
    -------
    card (int): the numeral value of the card drawn

    """
    card = rnd.choices(range(1, 11), weights = (3, 3, 3, 3, 3, 3, 3, 3, 3, 3))
    return int(card[0])

## Part 1.2.2 (13 points)
Its challenging to test a function which returns a random value, it gives different values by design!  However, the more samples we get from `draw_value()` the closer the fraction of values equal to 10 should be to our expected number of 10s output: 4/13 of the total cards (see also "Law of Large Numbers").

- Sample 13000 values from `draw_value()`
- record how many times you observe each card in a `defaultdict` named `card_count`
- print `card_count` and write one sentence in a markdown cell which explains why your results validate that `draw_value()` works


In [2]:
from collections import defaultdict

card_count = defaultdict(int)

for i in range(13000):
    card = draw_value()
    card_count[card] += 1

print(card_count)

defaultdict(<class 'int'>, {7: 1340, 9: 1300, 10: 1317, 8: 1287, 3: 1282, 4: 1304, 6: 1298, 2: 1272, 1: 1305, 5: 1295})


In [21]:
print(f"{round((card_count[10] / 13000) * 100, 4)}%")

9.8538%


The output suggests roughly **10%** of the cards drawn are 10s. Since we are choosing from a set of ten elements with replacement, the probability of choosing any one element out of the set will be **1/10**. This is consistent with the previous figure.

## Part 1.2.3 (15 points)

One of the big questions facing blackjack players is, "Given that my hand already has a value of X, what's the probability that the next card will bust my hand?".  For every starting hand sum from 10 to 21, we'll estimate this probability by simulating how often a hand goes bust among many randomly drawn hands.

- initialize `num_bust_per_start` as a defaultdict
    - keys are the starting hands from 10 to 21 (including both 10 and 21)
    - values (initially) count how many hands went bust (defaults to 0)
    - e.g. `num_bust_per_start = {19: 100, 15: 42}` indicates that, among all the simulations:
        - 100 hands starting at 19 went bust
        - 42 hands starting at 15 went bust
- for every `start_hand` from 10 to 21:
    - draw a single card via `draw_value()`
    - record the hand as a bust if the `start_hand` + `card_val` exceeds 21
    - repeat the above two steps so that you simulate 10,000 draws
        - 10,000 draws starting from 10
        - 10,000 draws starting from 11 
        - ... 


In [7]:
num_bust_per_start = defaultdict(int)
    
for i in range(10, 22):
    for a in range(10000):  
        start = draw_value()
        if i + start > 21:
            num_bust_per_start[i] += 1

## Part 1.2.4 (10 points)

Let's synthesize the results of part 1.2.3 immediately above by estimating the probability of going bust choosing to take 1 more card from every starting hand from 10 to 21

- make a new dictionary `prob_bust_given_start` whose keys are values 10 to 21 and values are probability estimates of going bust which correspond to `num_bust_per_start`
    - for example, if 7000 of the 10,000 draws starting from 15 went bust, the prob estimate of busting from 15 is .7
    
- print `prob_bust_given_start` so it may be observed
    - (optional: try using ["pretty print"](https://docs.python.org/3/library/pprint.html) to sort the output dictionary.  `from pprint import pprint` and then `pprint(prob_bust_given_tart)` on another line)

- A player wants to continue to add cards to their hand as long as the chance of going bust is no more than 50%.  What would you analysis suggest they do?  Write a one or two sentence response which is easily understood by everyone.


In [10]:
from pprint import pprint as p

prob_bust_per_start = defaultdict(int)
    
for item in num_bust_per_start:
    prob_bust_per_start[item] = round(((num_bust_per_start[item] / 10000) * 100), 2)
    
p(prob_bust_per_start)

defaultdict(<class 'int'>,
            {12: 9.21,
             13: 20.03,
             14: 29.79,
             15: 39.62,
             16: 49.58,
             17: 59.81,
             18: 70.52,
             19: 79.6,
             20: 89.92,
             21: 100.0})


As shown in the table above, the player should continue to add cards **as long as their starting value is lower than 16.** This way, their chance of going bust will always be equal to or less than 50%. 

# Triple-or-nothing
Triple-or-nothing is a game where a `jar` is initially set up with 1 coin and placed between two opposing players.  In each round:
- both players bet some fraction, `frac`
    - neither player knows the other's `frac` before choosing their own
    - fractions must be between 0 and 1 (including 0 and 1)
- coins are distributed to the players as:
    1. the player with the smaller $^1$ `frac` takes that fraction$^2$ of coins from the jar and places them in their `purse`
    1. the player with the larger$^1$ `frac` takes that fraction$^2$ of coins from the **remaining** jar and places them in their `purse`
    1. the "bank" (an infinite supply of coins) triples the value of the jar (fractional coins allowed)
    1. If the jar has no coins in it or the players have already played 10 rounds, the game ends
    
Each player seeks to earn as many coins as possible.

Footnote 1: You may assume the players do not write the same value down.  (see also: extra credit below)

Footnote 2: In the event the jar is emptied, the game ends


# Part 1.3 (16+2 pts autograder, 19 pts)

Complete the `update_round()` function below, which updates a round of triple-or-nothing as described above.  Your function should pass all the given assert statements below.

For (+2 pts) extra credit, revise your `update_round()` to support the case when players give the same `frac`.  In this event, the total number of coins taken out of the jar if the players went (in any order) is: 

    frac * jar + frac * (1 - frac) * jar = (2 - frac) * frac * jar

and it is split evenly between the players.

#### Hints:
- Study the docstring and testcases of `update_round()` before building it, make sure you understand why the test cases use the values they do
- Notice that `update_round()` does not know anything baout round indices, game stopping conditions etc


In [11]:
def update_round(jar, frac0, frac1):
    """ runs a round of triple-or-nothing
    
    arguments:
    -------
    jar (float): number of coins in the jar at round start
    frac0 (float): player0's fraction bet
    frac1 (float): player1's fraction bet
            
    returns:
    -------
    jar (float): number of coins in the jar at end of round
    new_coin0 (float): number of coins player0 has earned in this round
    new_coin1 (float): number of coins player1 has earned in this round
    
    """
    # check that both fraction inputs are valid
    assert 0 <= frac0 <= 1, 'invalid input: frac0'
    assert 0 <= frac1 <= 1, 'invalid input: frac1'
    
    # conduct round
    if frac0 == frac1:
        amt = ((2 - frac0) * frac0 * jar)
        new_coin0 = amt / 2
        new_coin1 = new_coin0
        jar -= amt
        
    elif frac0 < frac1:
        new_coin0 = frac0 * jar
        jar = jar - new_coin0
        new_coin1 = frac1 * jar
        jar = jar - new_coin1
        
    else: 
        new_coin1 = frac1 * jar
        jar = jar - new_coin1
        new_coin0 = frac0 * jar
        jar = jar - new_coin0
    
    jar *= 3
    return jar, new_coin0, new_coin1

In [14]:
assert update_round(jar=10, frac0=.1, frac1=0) == (27, 1, 0), '(2 pts)'
assert update_round(jar=10, frac0=.5, frac1=.2) == (12, 4, 2), '(2 pts)'
assert update_round(jar=10, frac0=1, frac1=.5) == (0, 5, 5), '(2 pts)'
assert update_round(jar=10, frac1=.9, frac0=1) == (0, 1, 9), '(2 pts)'

# swap frac0 / frac1 and new_coin0 / new_coin1 from previous 4 test cases
assert update_round(jar=10, frac1=.1, frac0=0) == (27, 0, 1), '(2 pts)'
assert update_round(jar=10, frac1=.5, frac0=.2) == (12, 2, 4), '(2 pts)'
assert update_round(jar=10, frac1=1, frac0=.5) == (0, 5, 5), '(2 pts)'
assert update_round(jar=10, frac1=1, frac0=.9) == (0, 9, 1), '(2 pts)'

# extra credit test cases
assert update_round(jar=10, frac0=1, frac1=1) == (0, 5, 5), '(1 pt)'
assert update_round(jar=20, frac0=.5, frac1=.5) == (15, 7.5, 7.5), '(1 pt)'

## Extra Credit (up to +4 points)

Program your own triple-or-nothing player.  Your new function must use the same inputs, outputs and name (`triple_nothing_player()`) as the function below for credit to be awarded.

- You needn't use all inputs provided, they're provided to help you build a player which is "smarter" or more fun
- You may, if you choose, call a working version of `update_round()` as defined above
- Do not `import` any other packages in building your `triple_nothing_player()` 

All student submissions will play one game against every other student submission.  The winner is the function with the largest coin total at the end of all games.

While technically, this is a competition, you're welcome to submit strategies which behave in funny ways without the intention of winning.  I'm hopeful that the data of all these games will be interesting to look at, if so we'll be seeing it again in DS2500.  To help identify which strategy is yours in the dataset, please output a unique (within the class) `name` to your strategy.  

All Extra Credit will be awarded based on the creativity, implementation and documentation of the submission. 


In [1]:
extra_credit_attempted = True

In [2]:
def triple_nothing_player(jar, frac_hist_opp, frac_hist_self):
    """ produces a fraction in a game of triple or nothing
    
    frac_hist objects are lists of previous fraction history in
    the game.  for example, frac_hist = [0, .01, .2] indicates
    a player had fraction 0 in the first round, .01 in the second
    and .2 in the third
    
    args:
    -------
        jar (float):value of the jar in current round
        frac_hist_opp (list): history of all fractions input by opposing player
        frac_hist_self (list): history of all fractions input by self
            
    returns:
    -------
        frac (float): fraction for current round
        name (str): a unique name given to your strategy
    """
    name = "Average Guess?"
    opp = sum(frac_hist_opp) / len(frac_hist_opp)
    user = sum(frac_hist_self) / len(frac_hist_self)
    frac = (user + opp) / 2
    return frac, name